# Langroid Implementation

langroid:
tokenizer: https://github.com/simonw/ttok/issues/8

In [1]:
# with open("private/hf-key", "r") as f:
#     key = f.read().rstrip("\n")
# import tokenizers
# from tokenizers import Tokenizer

# tokenizer = Tokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token=key)

import langroid as lr
import langroid.language_models as lm
from typing import Dict
from langroid.agent.chat_document import ChatDocument


def _select_choice(questions: Dict[str,str])->str:
    return list(questions.keys())[0]

llm_config = lm.OpenAIGPTConfig(
    chat_model="ollama/llama3.2",
    chat_context_length =20_000
)

agent_config = lr.ChatAgentConfig(
    llm=llm_config,
    system_message="You are a 25 year old athletic woman. Please answer the following questions.",
)



from typing import List, Optional
import langroid as lr
from langroid.agent.tool_message import ToolMessage
import langroid.language_models as lm

llm_config = lm.OpenAIGPTConfig(
    chat_model = "ollama/llama3.2",
    chat_context_length=16_00
)

config = lr.ChatAgentConfig(
    name="SurveyAgent",
    llm=llm_config,
    system_message="You are a 25 year old athletic female. Please answer the following survey questions related to fitness.",
    use_tools=True,
    use_functions_api=False,
    vecdb=None
)

class singleAnswerTool(lr.ToolMessage):
    request: str = "singleChoiceResponse"
    purpose: str = """Please read the provided options and select ONE answer that best applies to you. Only specify the <answer_key>, and not the description."""
    answer_key: int

    @classmethod
    def example(cls):
        return [
            (
                'How many times per week do you get groceries? {\'-8\': "I don\'t know", \'-7\': \'I prefer not to answer\', \'-1\': \'Appropriate skip\', \'1\': \'1 day a week\', \'2\': \'2 days a week\', \'3\': \'3 days a week\', \'4\': \'4 days a week\', \'5\': \'5 days a week\', \'6\': \'6 days a week\', \'7\': \'7 days a week\'}',
                cls(answer_key=1)
            )
        ]


class SurveyAgent(lr.ChatAgent):
    def __init__(self, config: lr.ChatAgentConfig):
        super().__init__(config)
        self.tool_expected = False
        self.answersIndices = []
        self.answers = []
        self.queued_question: dict


    def singleChoiceResponse(self, msg: singleAnswerTool ) -> str:
        # return a single possible answer from a set of questions
        return str(msg.answer_index)



    def llm_response(
        self, message: Optional[str | ChatDocument] = None
    ) -> Optional[ChatDocument]:
        self.tool_expected = True
        return super().llm_response(message)

    def user_response(
        self,
        msg: Optional[str | ChatDocument] = None,
    ) -> Optional[ChatDocument]:
        self.tool_expected = False
        return super().user_response(msg)

agent = SurveyAgent(config=config)
agent.enable_message(singleAnswerTool, use=True, handle=True)


2025-03-20 18:12:20 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                        using fake redis client


In [2]:
questions = {
    "-8": "I don't know",
    "-7": "I prefer not to answer",
    "-1": "Appropriate skip",
    "1": "1 day a week",
    "2": "2 days a week",
    "3": "3 days a week",
    "4": "4 days a week",
    "5": "5 days a week",
    "6": "6 days a week",
    "7": "7 days a week"
}

In [3]:
response = agent.llm_response(message=f"How many days per week do you work out? Please select one of the following: {questions}")

TOOL: singleChoiceResponse
            request: "How many days per week do you work out?"
            answer_key: 3

Stats: N_MSG=2, TOKENS: in=387, out=28, max=8192, ctx=1600, COST: now=$0.0000, cumul=$0.0000, tot=$0.0000 
(llama3.2)

In [6]:
agent.llm_response("Please tell us how you are feeling today?, please do not use any tools for this.")

Today I'm feeling great, a bit tired from my morning run, but overall energized and motivated to tackle the day!

Stats: N_MSG=8, TOKENS: in=502, out=25, max=8192, ctx=1600, COST: now=$0.0000, cumul=$0.0000, tot=$0.0000 
(llama3.2)

ChatDocument(content="Today I'm feeling great, a bit tired from my morning run, but overall energized and motivated to tackle the day!", metadata=ChatDocMetaData(source='LLM', source_content='context', is_chunk=False, id='5aea1aa1-02c2-40c2-840a-5085ef53a2a4', window_ids=[], parent_id='', child_id='', agent_id=FieldInfo(default=PydanticUndefined, default_factory=<function Agent.<lambda> at 0x11c668220>, extra={}), msg_idx=8, sender=<Entity.LLM: 'LLM'>, oai_tool_id=None, tool_ids=[], block=None, sender_name='', recipient='', usage=LLMTokenUsage(prompt_tokens=502, completion_tokens=25, cost=0.0, calls=0), cached=False, displayed=True, has_citation=False, status=None), reasoning='', content_any="Today I'm feeling great, a bit tired from my morning run, but overall energized and motivated to tackle the day!", oai_tool_calls=None, oai_tool_id2result=None, oai_tool_choice='auto', function_call=None, tool_messages=[], all_tool_messages=[], attachment=None)